In [1]:
#!/usr/bin/env python
# 04_clean_data.py
# Ian Flores Siaca & Betty Zhou November 23, 2018
#
# This script trains the Decision Tree Classifier.
# This script takes the input file path, and the output file path.
#
# Dependencies: pandas, sklearn, argparse, numpy
#
# Usage: python 04_clean_data.py "data/san_francisco_features.csv" "data/feature_results.csv"

In [5]:
# Loading the libraries to be used

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

import argparse

In [3]:
# Adding support to use script from the CLI

parser = argparse.ArgumentParser()
parser.add_argument('input_file')
parser.add_argument('output_file')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--input_file INPUT_FILE]
                             [--output_file OUTPUT_FILE]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-84e14869-3136-4dff-a07c-5b735ab68dbc.json


SystemExit: 2

/home/kropotkin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
# Parsing the CLI input to use it on Python

input_file = args.input_file
output_file = args.output_file

NameError: name 'args' is not defined

In [7]:
sf_data = pd.read_csv(input_file)
y = sf_data[['resolution']]
X = sf_data.drop('resolution', axis = 1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.50)

In [9]:
# Iterating thorugh different depths to find the optimal

parameters = {'max_depth': np.arange(1, 30, 5)}

tree = DecisionTreeClassifier(class_weight='balanced')
clf = GridSearchCV(tree, parameters, cv=3)
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': array([ 1,  6, 11, 16, 21, 26])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [10]:
best_parameters = clf.best_params_

In [11]:
# Training our model

tree = DecisionTreeClassifier(
    class_weight='balanced', max_depth=best_parameters['max_depth'])

tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=11, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [26]:
accuracy = tree.score(X_test, y_test)

In [12]:
feature_results = pd.DataFrame({
    'features': X.columns.tolist(),
    'feature_importance': tree.feature_importances_
})

feature_results = feature_results.sort_values(
    'feature_importance', ascending=False)
feature_results = feature_results.query('feature_importance > 0.01')

In [27]:
feature_results = pd.concat([pd.DataFrame({'features':['TEST_ACCURACY'], 
                        'feature_importance' : [accuracy]}), 
          feature_results])

,features,feature_importance
0,TEST_ACCURACY,0.813640
105,auto,0.193986
132,property,0.132385
38,category_VEHICLE THEFT,0.117373
125,mischief,0.087408
7,category_BURGLARY,0.072895
35,category_SUSPICIOUS OCC,0.068936
23,category_NON-CRIMINAL,0.043802
4,category_ASSAULT,0.043443
138,stolen,0.043177


In [ ]:
feature_results.to_csv(output_file)